Rock Paper Scissors is a classic hand game where players compete through using hand gestures (see [Wikipedia](https://en.wikipedia.org/wiki/Rock\_paper\_scissors)).  The Tensorflow-datasets package provides a 
library of hand gestures corresponding to allowed gestures in the game.  The original source is a synthetic data source, via Laurence Maroney ([source](https://laurencemoroney.com/datasets.html\#rock-paper-scissors-dataset)).

Write a convolutional neural network (CNN) that classifies these images, i.e., identifies the gesture (rock, paper, scissors) from each source image.  For convenience, this template is provided for loading the dataset and for data augmentation.  Feel free to make any changes to this template code.  Try to achieve a validation accuracy rate of 90\% or above.

You may make use of pretrained CNN architectures in Keras or develop your own.  If you develop your own, you are encouraged to review the network architectures discussed in class (e.g., LeNet and AlexNet).  Because the training dataset
is relatively small, you may need to employ techniques such as regularization and data augmentation to achieve a high accuracy rate.

Using a graphics card is highly recommended when fitting the model.  You can set Colab to use the graphics card in Runtime$\rightarrow$Manage Settings.

#Preamble

In [1]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from functools import partial
import numpy as np

Below are constants, configure as needed.

In [2]:
batch_size = 8   # size of mini-batch
image_size = 32  # size of image to feed into network
n_repeat = 1 # number of times to repeat the input dataset with distorted images

#Data loading and preprocessing

We first load the test dataset from tensorflow datasets (TFDS).  The original set has only training and validation sets, so we shuffle the dataset here to produce training, validation, and validation sets.

* [Documentation for `tfds.load`](https://www.tensorflow.org/datasets/api_docs/python/tfds/load)
* [Rock Paper Scissors dataset](https://www.tensorflow.org/datasets/catalog/rock_paper_scissors)
* [TFDS images in this dataset](https://knowyourdata-tfds.withgoogle.com/#tab=STATS&dataset=rock_paper_scissors)

In [3]:
(ds_orig, ds_val_orig, ds_test_orig), dsinfo = tfds.load(
    'rock_paper_scissors',
    split=("test+train[60%:]", "train[60%:80%]", "train[80%:]"),
    with_info=True,
    as_supervised=True,
    shuffle_files=False)

labelnames = dsinfo.features['label'].names

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteETSJL9/rock_paper_scissors-train.tfrecord


  0%|          | 0/2520 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/rock_paper_scissors/3.0.0.incompleteETSJL9/rock_paper_scissors-test.tfrecord


  0%|          | 0/372 [00:00<?, ? examples/s]

Dataset rock_paper_scissors downloaded and prepared to /root/tensorflow_datasets/rock_paper_scissors/3.0.0. Subsequent calls will reuse this data.


We create a convenience function, `show_sample_images` to show 10 random images from the set.  Example usage: `show_sample_images(ds_orig)`.

In [4]:
def show_sample_images(ds):
  fig, axs = plt.subplots(4, 4, figsize=(10,10))
  for ax, k in zip(np.array(axs).ravel(), ds.unbatch().shuffle(128).take(16)):
      print(k[0])
      print(k[1])
      ax.imshow(k[0])
      ax.set_title(labelnames[k[1]])
  fig.tight_layout()

We will need to perform some preprocessing.  The below preprocessing function is a starting point.

* `distort` is a network that performs image augmentation.  For input images, it will randomly provide distortions such as distorting contrast, zooming in or out, rotation, and translation.  Passing an image into this network will apply these distortions.  Refer to documentation for these functions [here](https://www.tensorflow.org/api_docs/python/tf/image).

* `preproc` is a preprocessing function that accepts an image and label, and can apply updates to both (similar to a FunctionTransformer in scikit).  We only apply distortions if augment is true (as we will see, we want to apply to the training dataset).  

   Preproc also converts the image to grayscale, which provides some performance improvement since color is not expected important for processing.  If you remove grayscale conversion, remember to change the depth of the input layer from 1 to 3.

  Finally, preproc rescales the image from 0-255 to 0-1 ([`convert_image_dtype`](https://www.tensorflow.org/api_docs/python/tf/image/convert_image_dtype)) and resizes the image.

  * We also create two wrapper functions, `preproc_no_distort` and `preproc_distort`, for later compatibility with the tensor flow's data object.

In [5]:
distort = keras.Sequential(
    [
    keras.layers.RandomContrast(0.2),
    keras.layers.RandomZoom((-.05, .05), fill_mode='nearest'),
    keras.layers.RandomRotation((-.04, .04), fill_mode='nearest'),
    keras.layers.RandomTranslation(0, (-.2, .2), fill_mode='nearest')
    ]
)
def preproc(image, label, augment):
  if augment:
    image = distort(image)
  image = tf.image.rgb_to_grayscale(image)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, (image_size, image_size))
  return image, label

def preproc_no_distort(image, label):
  return preproc(image, label, False)

def preproc_distort(image, label):
  return preproc(image, label, True)

We can now prepare the datasets.  For the three datasets defined above (training, validation, and test), we preprocess each element using the `map` function.  (See [guidelines for TensorFlow input pipelines](https://www.tensorflow.org/guide/data) and [`tf.data.Dataset` documentation](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).  From the texts, see also chapter 13 in Raschka and chapter 11 in Geron.  We also break the training set into minibatches.

You should not need to change this code (although you may want to change the mini-batch size above).

In [6]:
ds = ds_orig.map(preproc_distort).batch(batch_size)
ds_val = ds_val_orig.map(preproc_no_distort).batch(1)
ds_test = ds_test_orig.map(preproc_no_distort).batch(1)

# Build Model

In [11]:
# Write and compile the model here
# model = ...

from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

model = tf.keras.models.Sequential([

    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu',input_shape=(32,32,1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.1),  
    tf.keras.layers.Dense(128,activation='relu'),                               
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [12]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adagrad',
              metrics=['accuracy'])


# Fit

We have two callbacks#

* `ModelCheckpoint`, which saves the model after each mini-batch.  We set the `save_best_only` parameter so that if the model performance deteriorates, we only keep the weights from the best-performing batch.

* `EarlyStopping`, which stops the model after no improvement for four iterations.

Note that we repeat the dataset here by calling `ds.repeat(n_repeat)`.  Because we are using image augmentation, this allows the network to be trained on multiple variants of each distorted image.

**Remember to change the runtime type to GPU before running (Change Runtime Type under the Tools menu.)**  You will need to re-run the notebook after changing the runtime type.

In [ ]:
earlystopping = keras.callbacks.EarlyStopping(min_delta=0.005, patience=4)
checkpoint = keras.callbacks.ModelCheckpoint("mdl-rps-eval-gap.h5", save_best_only=True)
dsr = ds.repeat(n_repeat) if n_repeat > 0 else ds
history = model.fit(dsr, validation_data=ds_val, epochs=20,
                    callbacks=[earlystopping, checkpoint])


Epoch 1/20
173/173 [==============================] - 40s 226ms/step - loss: 0.4380 - accuracy: 0.8435 - val_loss: 0.3032 - val_accuracy: 0.8968
Epoch 2/20
173/173 [==============================] - 39s 223ms/step - loss: 0.4449 - accuracy: 0.8326 - val_loss: 0.2958 - val_accuracy: 0.9008
Epoch 3/20
173/173 [==============================] - 38s 219ms/step - loss: 0.4462 - accuracy: 0.8304 - val_loss: 0.2933 - val_accuracy: 0.9028
Epoch 4/20
173/173 [==============================] - 38s 222ms/step - loss: 0.4323 - accuracy: 0.8326 - val_loss: 0.2921 - val_accuracy: 0.9028
Epoch 5/20
173/173 [==============================] - 38s 222ms/step - loss: 0.4325 - accuracy: 0.8341 - val_loss: 0.2876 - val_accuracy: 0.9048
Epoch 6/20
173/173 [==============================] - 39s 224ms/step - loss: 0.4344 - accuracy: 0.8297 - val_loss: 0.2834 - val_accuracy: 0.9048
Epoch 7/20
173/173 [==============================] - 39s 223ms/step - loss: 0.4224 - accuracy: 0.8442 - val_loss: 0.2843 - val_ac

#  Evaluate

We evaluate the best-performing model (saved in the model checkpoint callback) against the test dataset.  Recall that we are looking for an accuracy of 0.9 or above.

In [ ]:
fin_model = keras.models.load_model("mdl-rps-eval-gap.h5")
_, accuracy = fin_model.evaluate(ds_test)
print(accuracy)

504/504 [==============================] - 3s 5ms/step - loss: 0.2858 - accuracy: 0.9107
0.9107142686843872
